# Implementación de la autoatención con pesos entrenables

En  esta  sección,  implementamos  el  mecanismo  de  autoatención  utilizado  en  la  arquitectura  original  del  transformador,  los  modelos  GPT  y  la  mayoría  de  los  demás  LLM  populares.  

Este  mecanismo  también  se  denomina  atención  escalar  de  producto  escalar.

![Texto alternativo](./imgs/3.12.png)

- La self-attention con pesos entrenables calcula vectores de contexto como combinaciones ponderadas de las entradas.

- La diferencia clave respecto a la versión básica: se introducen matrices de pesos entrenables que se ajustan durante el entrenamiento.

- Estas matrices permiten que el modelo aprenda a producir buenos vectores de contexto.

## Cálculo de los pesos de atención paso a paso

En este apartado se implementa el mecanismo de autoatención introduciendo tres matrices de peso entrenables: `Wq`, `Wk` y `Wv`. Estas matrices se utilizan para proyectar los *tokens* de entrada incrustados `x(i)` en vectores de **consulta (query)**, **clave (key)** y **valor (value)**, respectivamente. 

El procedimiento comienza calculando los vectores `q`, `k` y `v` mediante multiplicaciones de la entrada por las correspondientes matrices de peso. De forma análoga a lo visto en la sección anterior, primero se ilustra el cálculo de un único vector de contexto `z(i)`. Posteriormente, el método se generaliza para obtener todos los vectores de contexto de la secuencia.  

![Texto alternativo](./imgs/3.13.png)

In [3]:
import torch

#Considerar  la  siguiente  oración  de  entrada,  que  ya  ha  sido  incorporada  en  vectores  tridimensionales 
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your     
    [0.55, 0.87, 0.66], # journey  (x^2)
    [0.57, 0.85, 0.64], # starts   (x^3)
    [0.22, 0.58, 0.33], # with     
    [0.77, 0.25, 0.10], # one      
    [0.05, 0.80, 0.55]] # step     
)

In [4]:
x_2 = inputs[1] #segundo elemento de la entrada
d_in = inputs.shape[1] #tamaño de incrustación de entrada
d_out = 2 #tamaño de incrustación de salida

#Inicializar tres  matrices  de  peso  Wq ,  Wk  y  Wv

torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False) #reducir el desorden en las salidas
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

#Calcular los vectores de consulta, clave y valor 
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value

print(query_2) #vector bidimensional


tensor([0.4306, 1.4551])


Los  parámetros  de  peso  son  los  coeficientes  fundamentales  aprendidos  que  definen  las  
conexiones  de  la  red,  mientras  que  los  pesos  de  atención  son  dinámicos  y  específicos  del contexto.

Aunque el objetivo es z_2, z requiere de los vectores de clave y valor, para todos los elemnentos de entradas, ya que están involucrados en el cálculo.

In [5]:
keys = inputs @ W_key
values = inputs @ W_value
#6 tokens de entrada de un espacion de incrustacion 3D a uno 2D
print("Keys shape: ", keys.shape)
print("Values shape: ", values.shape)


Keys shape:  torch.Size([6, 2])
Values shape:  torch.Size([6, 2])


El segundo paso es calcular los puntuajes de atención

![Texto alternativo](./imgs/3.14.png)

El  cálculo  de  la  puntuación  de  atención  es  un  cálculo  de  producto  escalar  similar  al  utilizado  en  el  mecanismo  simplificado  de  autoatención.  La  novedad  radica  en  que  no  calculamos  directamente  el  producto  escalar  entre  los  elementos  de  entrada,  sino  que  utilizamos  la  consulta  y  la  clave  obtenidas  al  transformar  las  entradas  mediante  las respectivas  matrices  de  ponderación.

Primero calcula el puntuaje de atencio W_22

In [6]:
keys_2 = keys[1]
query_2 = x_2 @ W_query
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22) #resultado no normalizado

tensor(1.8524)


In [7]:
#generalizar calculo a todos los puntuajes de atención 
attn_score_2 = query_2 @ keys.T
print(attn_score_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


El tercer paso es pasar de los puntuajes de atentión a los pesos de atención

![Texto alternativo](./imgs/3.15.png)

Calcular  las  ponderaciones  de  atención  escalando  las  puntuaciones  de  atención  y  utilizando  la  función  softmax  que  utilizamos  anteriormente.  La  diferencia  con  la  función  
anterior  radica  en  que  ahora  escalamos  las  puntuaciones  de  atención  dividiéndolas  por  la  raíz  cuadrada  de  la  dimensión  de  incrustación  de  las  claves  (tenga  en  cuenta  que  calcular  la  raíz  cuadrada  equivale  matemáticamente  a  exponenciar  por  0,5).

In [8]:
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_score_2 / d_k**0.5, dim=-1) #Si la dimension es muy grande, los valores pueden ser grandes y softmaz se vuelvemuy picuda, ara elo se divide por la raiz de d_k
print(attn_weights_2)
print(keys.shape, d_k)
print(d_k ** 0.5)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])
torch.Size([6, 2]) 2
1.4142135623730951


El paso final es calcular los vectores de contexto

![Texto alternativo](./imgs/3.16.png)

De la misma forma que se calculo  el  vector  de  contexto  como  una  suma  ponderada  de  los  vectores  de  entrada,  ahora  se calcula  el  vector  de  contexto  como  una  suma  ponderada  de  los  vectores  de  valor. 

In [9]:
contex_vec_2 = attn_weights_2 @ values
print(contex_vec_2)

tensor([0.3061, 0.8210])


En la siguiente sección se generalizará este cálculo para z^T

### Consulta, Clave y Valor en el Mecanismo de Atención
Los términos “consulta” (query), “clave” (key) y “valor” (value) en los mecanismos de atención provienen del ámbito de la recuperación de información y bases de datos, donde se usan conceptos similares para almacenar, buscar y recuperar información.

- Consulta (Query):
Representa el elemento actual en el que el modelo se centra, por ejemplo, una palabra o token de una oración. Funciona como una consulta de búsqueda en una base de datos y se utiliza para explorar las demás partes de la secuencia de entrada, determinando cuánta atención se les debe prestar.

- Clave (Key):
Cada elemento de la secuencia de entrada tiene una clave asociada, que actúa como índice o identificador para localizar información relevante. Las claves permiten al modelo encontrar coincidencias con la consulta.

- Valor (Value):
Representa el contenido real o la representación de los elementos de entrada, similar al valor en un par clave-valor en una base de datos. Una vez que el modelo determina qué claves son más relevantes para la consulta, se recuperan los valores correspondientes, que contienen la información que se utilizará en la salida del mecanismo de atención.

Piensa en esto con un ejemplo concreto:

Secuencia: `["El", "gato", "come"]`

Cada palabra se transforma en **Q**, **K** y **V**.

Supongamos que estamos evaluando la palabra `"gato"` (**Q**).

Para decidir a qué palabras prestar atención:

- `"gato"` (**Q**) se compara con cada **Key**: `K("El")`, `K("gato")`, `K("come")`.
- La comparación (producto punto o coseno) nos da **qué tan relevante es cada token** respecto a `"gato"`.
- Luego usamos los valores correspondientes: `V("El")`, `V("gato")`, `V("come")` ponderados por esa relevancia.

La analogía con SQL ayuda a entenderlo así:

```sql
SELECT valor FROM tabla WHERE key = query;


## Implementación de una clase compacta de Python con autoatención

Para una futura implementación de una LLM resulta útil organizar este código en una clase Python.


In [10]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))
    
    def forward(self, x):
        keys = x @ self.W_key
        querys = x @ self.W_query
        values = x @ self.W_value
        
        attn_scores = querys @ keys.T #omega
        attn_weights = torch.softmax(attn_scores/(keys.shape[-1]**0.5), dim=-1)
        context_vec = attn_weights @ values

        return context_vec

El método __init__ se inicializan las matrices de pesos entrenables, cada una transaformando cada dimensión d entrada d_in en una dimensión de salida d_out

Durante  el  paso  hacia  adelante,  utilizando  el  método  forward,  se claculan  los  puntajes  de  atención  (attn_scores)  multiplicando  consultas y  claves,  normalizando  estos  puntajes  usando  softmax.
Finalmente,  se crea  un  vector  de  contexto  ponderando  los  valores  con  estas  atenciones  normalizadas.


In [11]:
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in=d_in, d_out=d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


![Texto alternativo](./imgs/3.17.png)

En  la  autoatención,  transformamos  los  vectores  de  entrada  de  la  matriz  de  entrada  X  con  las  tres  matrices  de  
ponderación:  Wq,  Wk  y  Wv.  A  continuación,  calculamos  la  matriz  de  ponderación  de  la  atención  a  partir  de  las  consultas  (Q)  y  las  claves  (K)  resultantes.  Utilizando  las  ponderaciones  y  valores  de  la  atención  (V),  calculamos  los  vectores  de  contexto  (Z).  (Para  nortemayor  claridad  visual,  en  esta  figura  nos  centramos  en  un  único  texto  de  entrada  con  tokens,  no  en  un  conjunto  de  múltiples  entradas.  

Por  consiguiente,  el  tensor  de  entrada  3D  se  simplifica  a  una  matriz  2D  en  este  contexto.  Este  enfoque  permite  una visualización  y  comprensión  más  sencillas  de  los  procesos  involucrados.  Además,  para  mantener  la  coherencia  con  las  figuras  posteriores,  los  valores  de  la  matriz  de  atención  no  representan  las  ponderaciones  reales  de  la  atención)

La  autoatención  implica  las  matrices  de  pesos  entrenables  Wq ,  Wk  y  Wv .  Estas  matrices  transforman  los  datos  de  entrada  en  consultas,  claves  y  valores,  componentes  cruciales  del  mecanismo  de  atención.  A  medida  que  el  modelo  se  expone  a  más  datos  
durante  el  entrenamiento,  ajusta  estos  pesos  entrenables.

Se puede mejorar  aún  más  la  implementación  de  SelfAttention_v1  utilizando  las  capas  nn.Linear  de  PyTorch ,  que  realizan  la  multiplicación  de  matrices  eficazmente  cuando  las  unidades  de  sesgo  están  deshabilitadas.  Además,  una  ventaja  significativa  de  usar  nn.Linear  en  lugar  de  implementar  manualmente  nn.Parameter(torch.rand(...))  es  que  nn.Linear  cuenta  con  un  esquema  de  inicialización de  pesos  optimizado,  lo  que  contribuye  a  un  entrenamiento  del  modelo  más  estable  y  eficaz.

In [14]:
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
    
    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores =queries@keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        context_vec = attn_weights @ values
        return context_vec

In [16]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in=d_in, d_out=d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


SelfAttention_v1  y  SelfAttention_v2  dan  resultados  diferentes  porque  utilizan  pesos  iniciales  diferentes  para  las  matrices  de  peso,  ya  que  nn.Linear  utiliza  un  esquema  de  inicialización  de  peso  más  sofisticado.

nn.Linear  en  SelfAttention_v2  utiliza  un  esquema  de  inicialización  de  pesos  diferente  al  de  nn.Parameter(torch.rand(d_in,  d_out))  en  SelfAttention_v1,  lo  que  provoca  que  ambos  mecanismos  produzcan  resultados  distintos.  Para  comprobar  que  ambas  implementaciones,  SelfAttention_v1  y  SelfAttention_v2,  son  similares,  podemos  transferir  las  matrices  de  pesos  de  un  objeto  SelfAttention_v2  a  un  objeto  SelfAttention_v1,  de  modo  que  ambos  objetos  produzcan  los  mismos  resultados

In [ ]:
sa_v1.W_query.data = sa_v2.W_query.weight.data.T.clone()
sa_v1.W_key.data   = sa_v2.W_key.weight.data.T.clone()
sa_v1.W_value.data = sa_v2.W_value.weight.data.T.clone()

torch.manual_seed(789)
print(sa_v1(inputs))


tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


[Ocultación de palabras futuras con atención casual](./5_ocultacion_palabras_futuras.ipynb)